Let's set up our data

The above dataset includes the ORCHARD column, which is 1 if the house is on Orchard and 0 otherwise.  Binary variables like this can be very effective, since it allows us to examine the marginal value of a single factor (in this case, having a house on Orchard).

But what about the other streets?  What are the relative effects of houses on Apple, Townsend, and the other streets?  How do you handle regression with categorical variables, when there are more than two categories?

The answer is to use multiple columns.  For example, assume you have three streets, Apple, Beverly, and Carousel.  Assume the data look like this:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('Streets.csv')
df.head()

,ID,STREETNO,STREET,ZESTIMATE,SQFT,BEDR,BATHR,YRBUILT,LOTSIZE,SOLDFOR,YRSOLD
0,150,504,Apple,244069,1960,3,2.0,1400,0.36,NaN,NaN
1,23,120,Townsend,303508,1800,3,3.0,1903,0.35,183000.0,1997.0
2,95,56,Sunset,307445,2186,4,3.0,1915,0.26,532000.0,2007.0
3,99,45,Sunset,297546,1625,3,1.5,1915,0.27,NaN,NaN
4,10,208,Orchard,364482,2125,5,4.0,1921,0.53,280000.0,1995.0


I simplified the Old Newark dataset to make it easier to understand.  Let's confirm our streets by running a frequency table:

In [2]:
df.STREET.value_counts()

Orchard     4
Sunset      3
Townsend    1
Apple       1
Name: STREET, dtype: int64

In our previous example, we had only two values for the ORCHARD column: 1 and 0. We also needed only one column to handle the fact that we were examining Orchard vs non-Orchard houses.  In a similar manner, if you have n different values, you need only n-1 columns.  Since we have four different streets in our reduced dataframe, we need only three columns to handle it.  

Pandas also have an awesome function that handles the encoding for us: get_dummies. The following code generates the three needed columns that are necessary to track the four different streets:

In [3]:
df2 = df.copy()
df2 = pd.get_dummies(df2)

In [4]:
df2.head()

,ID,STREETNO,ZESTIMATE,SQFT,BEDR,BATHR,YRBUILT,LOTSIZE,SOLDFOR,YRSOLD,STREET_Apple,STREET_Orchard,STREET_Sunset,STREET_Townsend
0,150,504,244069,1960,3,2.0,1400,0.36,NaN,NaN,1,0,0,0
1,23,120,303508,1800,3,3.0,1903,0.35,183000.0,1997.0,0,0,0,1
2,95,56,307445,2186,4,3.0,1915,0.26,532000.0,2007.0,0,0,1,0
3,99,45,297546,1625,3,1.5,1915,0.27,NaN,NaN,0,0,1,0
4,10,208,364482,2125,5,4.0,1921,0.53,280000.0,1995.0,0,1,0,0


Notice how the df2 dataframe now has four new columns, each of which are coded to indicate a specific street.  STREET_Apple, for example, has a value of 1 when the street is Apple, and a value of 0 when it is not. The other streets are similar.

I mentioned that you only need three columns when you have four different categories.  Assume that STREET_Townsend does not exist.  The first row indicates Apple, the third and fourth rows are Sunset, and the last row is Orchard.  How do we know then when a house is on Townsend?  The values for the three columns are all zero (row 2).  

**When doing regression with these encoded columns, it is critical that you do NOT include all four columns!**  Why? The combination of the four columns will result in strong multicollinearity, since all together they are perfectly correlated with each other.  

Let's run a regression that includes only the four columns we just derived:

In [5]:
# split data into X and Y dataframes
X = df2[['STREET_Apple', 'STREET_Orchard',
       'STREET_Sunset', 'STREET_Townsend']].copy()
Y = df2['ZESTIMATE'].copy()

In [7]:
# Run regression using statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math
X = sm.add_constant(X) # required if constant expected
est = sm.OLS(Y,X).fit() # fit model


Now let's print out the VIF statistics:

In [7]:
variables = est.model.exog # get model features
vif = pd.DataFrame() # create a dataframe
vif["VIF Factor"] = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif["features"] = X.columns
print('VIF: {}'.format(vif))

VIF:    VIF Factor         features
0         NaN            const
1         inf     STREET_Apple
2         inf   STREET_Orchard
3         inf    STREET_Sunset
4         inf  STREET_Townsend


C:\Users\Mark\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Mark\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Note that the VIF scores are "inf", which means infinity.  With a perfect correlation (1) in the VIF formula, the result is a divided by zero error, which results in an infinite VIF level.

So instead, we choose a reference street, and include all the OTHER variables.  For example, if you want to see the value of owning a house relative to one on Townsend, you would include the other columns:

In [8]:
# split data into X and Y dataframes
X = df2[['STREET_Apple', 'STREET_Orchard',
       'STREET_Sunset', 'SQFT']].copy()
Y = df2['ZESTIMATE'].copy()

In [9]:
# Run regression using statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math
X = sm.add_constant(X) # required if constant expected
est = sm.OLS(Y,X).fit() # fit model
predictions = est.predict() # get predicted values
print(est.summary()) # prints full regression results
print("\nAverage error: {:.2f}.".format(math.sqrt(est.mse_resid)))

                            OLS Regression Results                            
Dep. Variable:              ZESTIMATE   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     3.047
Date:                Wed, 11 Nov 2020   Prob (F-statistic):              0.153
Time:                        11:56:33   Log-Likelihood:                -99.123
No. Observations:                   9   AIC:                             208.2
Df Residuals:                       4   BIC:                             209.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.431e+05   5.98e+04      4.

C:\Users\Mark\anaconda3\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


The above results indicate that relative to Townsend, a house on:
* Orchard is worth \$15160 more (but it is not significant)
* Apple is worth \$64800 less (and it is significant)
* Sunset is woth \$169 more (and it is not significant)

You should also note the disparity between the R2 and the adjusted R2.  This suggests that the model is overfit.

In [27]:
variables = est.model.exog # get model features
vif = pd.DataFrame() # create a dataframe
vif["VIF Factor"] = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif["features"] = X.columns
print('VIF: {}'.format(vif))

VIF:    VIF Factor        features
0   66.389637           const
1    1.822563    STREET_Apple
2    2.931535  STREET_Orchard
3    2.686696   STREET_Sunset
4    1.251259            SQFT


Even without the third column, the VIF results suggest the presence of multicollinearity.  